In [1]:
## Import Common Package

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [1]:
import xgboost

In [21]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from xgboost import XGBRegressor, XGBClassifier

conda install -c anaconda py-xgboost
conda install -c anaconda xgboost

In [4]:
from sklearn.svm import SVC, SVR

In [ ]:
SVC ==> C = Classifier => untuk Klasifikasi
SVR ==> R = Regressor => Untuk Regresi

In [6]:
## Import Data

In [7]:
df = pd.read_csv('life_expectancy.csv')
df.head()

,country,year,status,life_expectancy,adult_mortality,infant_deaths,alcohol,percentage_expenditure,HepB,measles,...,Polio,total_expenditure,DPT,HIV_AIDS,GDP,population,thinness_10_19,thinness_5_9,HDI,school_year
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 22 columns):
country                   2938 non-null object
year                      2938 non-null int64
status                    2938 non-null object
life_expectancy           2928 non-null float64
adult_mortality           2928 non-null float64
infant_deaths             2938 non-null int64
alcohol                   2744 non-null float64
percentage_expenditure    2938 non-null float64
HepB                      2385 non-null float64
measles                   2938 non-null int64
BMI                       2904 non-null float64
u5_deaths                 2938 non-null int64
Polio                     2919 non-null float64
total_expenditure         2712 non-null float64
DPT                       2919 non-null float64
HIV_AIDS                  2938 non-null float64
GDP                       2490 non-null float64
population                2286 non-null float64
thinness_10_19            2904 non-null

In [9]:
df.isna().sum()

country                     0
year                        0
status                      0
life_expectancy            10
adult_mortality            10
infant_deaths               0
alcohol                   194
percentage_expenditure      0
HepB                      553
measles                     0
BMI                        34
u5_deaths                   0
Polio                      19
total_expenditure         226
DPT                        19
HIV_AIDS                    0
GDP                       448
population                652
thinness_10_19             34
thinness_5_9               34
HDI                       167
school_year               163
dtype: int64

In [10]:
df.dropna(inplace=True)

In [11]:
df.isna().sum()

country                   0
year                      0
status                    0
life_expectancy           0
adult_mortality           0
infant_deaths             0
alcohol                   0
percentage_expenditure    0
HepB                      0
measles                   0
BMI                       0
u5_deaths                 0
Polio                     0
total_expenditure         0
DPT                       0
HIV_AIDS                  0
GDP                       0
population                0
thinness_10_19            0
thinness_5_9              0
HDI                       0
school_year               0
dtype: int64

In [13]:
df.country.nunique()

133

In [14]:
### Rough Data Preparation

In [15]:
df = pd.get_dummies(df, columns=['country', 'status'] )

In [16]:
df.head()

,year,life_expectancy,adult_mortality,infant_deaths,alcohol,percentage_expenditure,HepB,measles,BMI,u5_deaths,...,country_Turkmenistan,country_Uganda,country_Ukraine,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Zambia,country_Zimbabwe,status_Developed,status_Developing
0,2015,65.0,263.0,62,0.01,71.279624,65.0,1154,19.1,83,...,0,0,0,0,0,0,0,0,0,1
1,2014,59.9,271.0,64,0.01,73.523582,62.0,492,18.6,86,...,0,0,0,0,0,0,0,0,0,1
2,2013,59.9,268.0,66,0.01,73.219243,64.0,430,18.1,89,...,0,0,0,0,0,0,0,0,0,1
3,2012,59.5,272.0,69,0.01,78.184215,67.0,2787,17.6,93,...,0,0,0,0,0,0,0,0,0,1
4,2011,59.2,275.0,71,0.01,7.097109,68.0,3013,17.2,97,...,0,0,0,0,0,0,0,0,0,1


In [17]:
#### Splitting Dataset

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X = df.drop(columns='life_expectancy')
y = df.life_expectancy

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = .8, random_state=42)

In [23]:
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor

In [24]:
#### Base Model

In [ ]:
### Base XGB

In [61]:
base_XGB = XGBRegressor()

In [62]:
base_XGB.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [63]:
y_base_XGB = base_XGB.predict(X_test)

In [64]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [66]:
r2_base_XGB = r2_score(y_test, y_base_XGB)
r2_base_XGB

0.9310535388609258

In [75]:
MAE_base_XGB = mean_absolute_error(y_test, y_base_XGB)

In [76]:
MSE_base_XGB = mean_squared_error(y_test, y_base_XGB)

In [77]:
RMSE_base_XGB = np.sqrt(MSE_base_XGB)

In [78]:
### Base SVM

In [80]:
base_SVM = SVR()

In [81]:
base_SVM.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

In [82]:
y_base_SVM = base_SVM.predict(X_test)

In [83]:
r2_base_SVM = r2_score(y_test, y_base_SVM)
r2_base_SVM

-0.023866462584022807

In [84]:
MAE_base_SVM = mean_absolute_error(y_test, y_base_SVM)

In [85]:
MSE_base_SVM = mean_squared_error(y_test, y_base_SVM)

In [86]:
RMSE_base_SVM = np.sqrt(MSE_base_SVM)

In [67]:
#### Base Random Forest

In [68]:
from sklearn.ensemble import RandomForestRegressor

In [69]:
base_RF = RandomForestRegressor()

In [70]:
base_RF.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [71]:
y_RF = base_RF.predict(X_test)

In [72]:
r2_base_RF = r2_score(y_test, y_RF)
r2_base_RF

0.9402503429454956

In [73]:
from sklearn.linear_model import LinearRegression

In [74]:
model = LinearRegression().fit(X_train, y_train)
model.score(X_test, y_test)

0.9451343320896336

In [87]:
#### Hyper Parameter Tuning

In [88]:
## XGB Hyper Parameter

In [89]:
XGB_model = XGBRegressor()

In [ ]:
max_depth=3, learning_rate=0.1, n_estimators=100, gamma=0, colsample_bytree=1, subsample=1, reg_alpha=0, reg_lambda=1,

In [94]:
10*10*10*10*10*10*10*10*4

400000000

In [ ]:
max_depth = nilainya integer (kedalaman pohon) ==> jangan kebanyakan
learning_rate = nilai nya float, pake logspace ==> Kecepatan/durasi belajar
n_estimators = jumlah pohon , nilai ratusan ==> jangan kebanyakan juga 
gamma = min_impurity_decrease ==> minimal error yg harus didapatkan biar pohon ngesplit , nilainya integer
colsample_bytree = max_features nya RF, pake presentase 0.1 - 1
subsample = mirip sama max features tapi untuk BARIS, pake presentase juga 0.1 - 1
Regularization
reg_alpha=0, ==> L1 => Lasso ==> float ==> logspace
reg_lambda=1 ==> L2 => Ridge ==> float ==> Logspace

In [ ]:
param_xgb = {
    "max_depth" = [2, 3, 5, 7, 9]
}

In [ ]:
5 * 5